In [83]:
#Imports
import torch
import torchvision
import numpy as np
import torch.nn as nn
import torch.functional as F
import pandas as pd
import os
from torchvision.io import read_image
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision.transforms import transforms
import matplotlib.pyplot as plt

In [84]:
#Constants
img_dir = r"C:\Users\Humperdink2\Documents\github\NewHacks2023\Food Images\Food Images"
data_dir = r"C:\Users\Humperdink2\Documents\github\NewHacks2023\Food Ingredients and Recipe Dataset with Image Name Mapping.csv"
batch_size = 128
learning_rate = 0.001
momentum = 0.9
img_size = 169
output_size = 13500

In [94]:
# constructing the dataloader
class foodImageDataset(Dataset):
    def __init__(self, img_dir, data_dir, transformation = None):
        self.img_dir = img_dir
        #self.ingr = pd.read_csv(data_dir, header=1,usecols=[5])
        self.transform = transformation
        self.labels = pd.read_csv(data_dir, header=1,usecols=[1,4])

    #Later augment the dataset to give more images per class to increase accuracy

    def __len__(self):
       return len(self.labels)
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.labels.iloc[idx,1] + ".jpg")
        image = read_image(img_path)
        label = self.labels.iloc[idx, 0]
        if self.transform is not None:
            image = self.transform(image)
        return image, label

    # def __getingrs__(self, index):
    #     return self.ingr[index]

In [105]:
#Model Declaration
class FoodModel(nn.Module):
    def __init__(self,img_size, output_size):
        super().__init__()
        self.conv1 = nn.Conv2d(3,55,kernel_size=(9,9), stride=3, padding=1)
        self.act1 = nn.ReLU()
        self.drop1 = nn.Dropout(0.3)
        print("init1")
        self.conv2 = nn.Conv2d(55,55,kernel_size=(3,3), stride=1, padding=1)
        self.act2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=(2,2))
        print("init2")
        self.flat = nn.Flatten()
        print("init3")
        self.fc3 = nn.Linear(int(55*55*55/4), int(output_size*2))
        self.act3 = nn.ReLU()
        self.drop3 = nn.Dropout(.5)
        print("init4")
        self.fc4 = nn.Linear(output_size*2, output_size)
        print("init5")
    def forward(self, image):
        #Transforming to Square
        image = transforms.Resize(169)
        print("train1")
        #First layer
        image = self.act1(self.conv1(image))
        image = self.drop1(image)
        print("train2")
        #Second layer
        image = self.act2(self.conv2(image))
        image = self.pool2(image)
        print("train3")
        #Third layer
        image = self.flat(image)
        image = self.act3(self.fc3(image))
        image = self.drop3(image)
        print("train4")
        #output
        image = self.fc4(image)
        return image
    
    def train(self, epochs, train_dl, loss_fn, opt):
        for epoch in range(epochs):
            count = 0
            for images, labels in train_dl:
                print("picture: ", count)
                preds = self(images)
                loss = loss_fn(preds, labels)
                #training
                opt.zero_grad()
                loss.backward()
                opt.step()
                count += 1

In [92]:
model = FoodModel(img_size, output_size)

1
2
3
4
5


In [88]:
opt = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [111]:
#Setting Model
loss_fn = nn.CrossEntropyLoss()
transform = transforms.Compose([
    transforms.Resize((169,169)),
    transforms.ToTensor(),
])
food = foodImageDataset(img_dir, data_dir, transformation=transform)
food_dl = DataLoader(food, batch_size, shuffle=True)

In [112]:
#Running Model
model.train(5,food_dl,loss_fn,opt)

c:\Users\Humperdink2\anaconda3\envs\newhacks\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


TypeError: pic should be PIL Image or ndarray. Got <class 'torch.Tensor'>

In [ ]:
def get(img_dir, data_dir, transformation = None):
        data = pd.read_csv(data_dir, header=1)
        
        # Extracting specific columns for labels and ingredients
        labels = data.iloc[:, [1,4]]  # Assuming the label is in column 1 and ingredient in column 4
        print(labels)
        ingr = data.iloc[:, 5]
        print(ingr[6957, 1])

get(img_dir,data_dir)